In [1]:
import pandas as pd
import yaml
import numpy as np
from datetime import date
import os
from src.sim_utils import *

Open default parameters

In [2]:
defaultParamPath = "../configs/default-manc-pulse.yaml"
params = load_from_yaml(defaultParamPath)

Change defaults

This example: Want to run *n* simulations of varying stim strengths and random seeds while activating both BDN2s

In [3]:
simStr = "mancBDN2activation"

nSims = 20
params["paramsToIterate"] = ["stimI","seed"]

# get BDN2 neurons
wTable = pd.read_csv(params["dfPath"],index_col=0)
stimNeurons = wTable.loc[wTable["type"] == "DNxl058"]
params["stimNeurons"] = stimNeurons.index.to_list()

# set different stimIs and seeds for each
meanStimI = np.mean((stimNeurons["size"]/512)/60000)
stdStimI = meanStimI/10
params["seed"] = np.random.default_rng().integers(10000,size=nSims).tolist()
params["stimI"] = np.random.default_rng().normal(meanStimI,stdStimI,size=nSims).tolist()

Save the new config file that corresponds to this set of simulations

In [4]:
dateStr = date.today().strftime(format="%Y%m%d")
saveStr = f"{dateStr}_{simStr}"
params["name"] = saveStr

configPath = save_config(params,f"../configs/{saveStr}.yaml")

**Here is where you would run the simulations, using `run_from_config.py`.**

The old way looked like this:
```
for i in range(nSims):
    os.system(f"python {os.path.abspath('../src/run_from_config.py')} '{configPath}' --idx {i}")
```

But you would actually want to run using Jax!